In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from sionna.constants import GLOBAL_SEED_NUMBER

import tensorflow as tf
from tensorflow.keras.layers import Layer
from my_code.mysionna.utils import expand_to_rank,complex_normal

class AWGN(nn.Module):
    r"""AWGN(dtype=torch.complex64, **kwargs)

    Add complex AWGN to the inputs with a certain variance.

    This class inherits from the PyTorch `nn.Module` class and can be used as layer in
    a PyTorch model.

    This layer adds complex AWGN noise with variance ``no`` to the input.
    The noise has variance ``no/2`` per real dimension.
    It can be either a scalar or a tensor which can be broadcast to the shape
    of the input.

    Example
    --------

    Setting-up:

    >>> awgn_channel = AWGN()

    Running:

    >>> # x is the channel input
    >>> # no is the noise variance
    >>> y = awgn_channel((x, no))

    Parameters
    ----------
        dtype : Complex torch.dtype
            Defines the datatype for internal calculations and the output
            dtype. Defaults to `torch.complex64`.

    Input
    -----

        (x, no) :
            Tuple:

        x :  Tensor, torch.complex
            Channel input

        no : Scalar or Tensor, torch.float
            Scalar or tensor whose shape can be broadcast to the shape of ``x``.
            The noise power ``no`` is per complex dimension. If ``no`` is a
            scalar, noise of the same variance will be added to the input.
            If ``no`` is a tensor, it must have a shape that can be broadcast to
            the shape of ``x``. This allows, e.g., adding noise of different
            variance to each example in a batch. If ``no`` has a lower rank than
            ``x``, then ``no`` will be broadcast to the shape of ``x`` by adding
            dummy dimensions after the last axis.

    Output
    -------
        y : Tensor with same shape as ``x``, torch.complex
            Channel output
    """

    def __init__(self, dtype=torch.complex64, **kwargs):
        super().__init__(**kwargs)
        if dtype == torch.complex32:
            self._real_dtype =  torch.float16
        if dtype == torch.complex64:
            self._real_dtype = torch.float32
        if dtype == torch.complex128:
            self._real_dtype = torch.float64

    def forward(self, inputs):
        x, no = inputs

        # Create tensors of real-valued Gaussian noise for each complex dim.
        noise = complex_normal(x.shape, dtype=x.dtype)

        # Add extra dimensions for broadcasting
        no = expand_to_rank(no, x.ndim, axis=-1)
        
        # Apply variance scaling
        no = no.to(self._real_dtype)
        noise *= torch.sqrt(no).to(noise.dtype)

        # Add noise to input
        y = x + noise

        return y

# Test the AWGN layer
def test_awgn():
    # Create AWGN layer instance
    awgn_layer = AWGN()

    # Define input data x (batch_size=5, num_symbols=13, complex dimension=4)
    x_real = torch.randn(5, 13, 4, dtype=torch.float32)
    x_imag = torch.randn(5, 13, 4, dtype=torch.float32)
    x = torch.complex(x_real, x_imag)

    # Define noise power no (shape can be broadcasted to x)
    no = torch.tensor([0.1], dtype=torch.float32)

    # Call forward method
    y = awgn_layer((x, no))

    # Print outputs
    print("Input x shape:", x.shape)
    print("Noise power no shape:", no.shape)
    print("Output y shape:", y.shape)
    print("Output y:", y)

# Run the test
test_awgn()

Input x shape: torch.Size([5, 13, 4])
Noise power no shape: torch.Size([1])
Output y shape: torch.Size([5, 13, 4])
Output y: tensor([[[ 2.8106-0.1870j, -2.0622+0.5667j,  0.3301+1.3357j,  1.2612-2.3521j],
         [ 1.6770-0.3806j, -0.0386+0.8465j, -0.5338-1.2473j, -1.3280+0.5513j],
         [-0.1246+0.9492j, -0.1742+1.9775j,  0.6104+0.5461j, -0.6811-0.2520j],
         [-0.0535+0.9818j, -1.2457-0.9112j,  2.6252-0.1438j, -1.7111+0.9850j],
         [ 1.9797+0.9551j,  1.5705+0.1831j,  0.3246-0.5983j,  0.5643+0.0730j],
         [ 0.4130+0.3080j,  0.7996-1.4629j,  0.5138+1.7670j, -0.3706-0.5336j],
         [-1.9767+0.9543j,  0.2290+1.4158j, -1.7848-0.6067j,  1.3560+0.2480j],
         [ 0.3814+0.0196j, -0.6307+0.7764j, -0.0994-1.5214j,  0.4858-0.1257j],
         [ 0.3309+1.1302j, -1.3878+0.5040j, -1.4665+1.7959j, -0.1855+1.2231j],
         [-2.6231+0.7559j, -1.2976-0.3810j,  0.5804+0.6925j,  1.4915+0.9617j],
         [ 0.6169-0.1600j,  0.8598+0.5025j, -1.3022+0.9073j, -1.7532+0.0791j],
      

# tensorflow

In [14]:
#
# SPDX-FileCopyrightText: Copyright (c) 2021-2023 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
# SPDX-License-Identifier: Apache-2.0
#
"""Layer for simulating an AWGN channel"""

import tensorflow as tf
from tensorflow.keras.layers import Layer
from sionna.utils import expand_to_rank, complex_normal

from sionna.constants import GLOBAL_SEED_NUMBER

class AWGN(Layer):
    r"""AWGN(dtype=tf.complex64, **kwargs)

    Add complex AWGN to the inputs with a certain variance.

    This class inherits from the Keras `Layer` class and can be used as layer in
    a Keras model.

    This layer adds complex AWGN noise with variance ``no`` to the input.
    The noise has variance ``no/2`` per real dimension.
    It can be either a scalar or a tensor which can be broadcast to the shape
    of the input.

    Example
    --------

    Setting-up:

    >>> awgn_channel = AWGN()

    Running:

    >>> # x is the channel input
    >>> # no is the noise variance
    >>> y = awgn_channel((x, no))

    Parameters
    ----------
        dtype : Complex tf.DType
            Defines the datatype for internal calculations and the output
            dtype. Defaults to `tf.complex64`.

    Input
    -----

        (x, no) :
            Tuple:

        x :  Tensor, tf.complex
            Channel input

        no : Scalar or Tensor, tf.float
            Scalar or tensor whose shape can be broadcast to the shape of ``x``.
            The noise power ``no`` is per complex dimension. If ``no`` is a
            scalar, noise of the same variance will be added to the input.
            If ``no`` is a tensor, it must have a shape that can be broadcast to
            the shape of ``x``. This allows, e.g., adding noise of different
            variance to each example in a batch. If ``no`` has a lower rank than
            ``x``, then ``no`` will be broadcast to the shape of ``x`` by adding
            dummy dimensions after the last axis.

    Output
    -------
        y : Tensor with same shape as ``x``, tf.complex
            Channel output
    """

    def __init__(self, dtype=tf.complex64, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        self._real_dtype = tf.dtypes.as_dtype(self._dtype).real_dtype

    def call(self, inputs):
        tf.random.set_seed(GLOBAL_SEED_NUMBER)

        x, no = inputs

        # Create tensors of real-valued Gaussian noise for each complex dim.
        noise = complex_normal(tf.shape(x), dtype=x.dtype)

        # Add extra dimensions for broadcasting
        no = expand_to_rank(no, tf.rank(x), axis=-1)

        # Apply variance scaling
        no = tf.cast(no, self._real_dtype)
        noise *= tf.cast(tf.sqrt(no), noise.dtype)

        # Add noise to input
        y = x + noise

        return y
# 创建测试函数
def test_awgn():
    tf.random.set_seed(GLOBAL_SEED_NUMBER)
    # 创建 AWGN 类实例
    awgn_layer = AWGN()

    # 定义输入数据 x (batch_size=5, num_symbols=13, complex dimension=4)
    x_real = tf.random.normal((5, 13, 4))
    x_imag = tf.random.normal((5, 13, 4))
    x = tf.complex(x_real, x_imag)

    # 定义噪声功率 no (形状可以广播到 x)
    no = tf.constant([0.1], dtype=tf.float32)

    # 调用 call 方法
    y = awgn_layer((x, no))

    # 打印输出
    print("Input x shape:", x.shape)
    print("Noise power no shape:", no.shape)
    print("Output y shape:", y.shape)
    print("Output y:", y)

# 运行测试函数
test_awgn()

Input x shape: (5, 13, 4)
Noise power no shape: (1,)
Output y shape: (5, 13, 4)
Output y: tf.Tensor(
[[[ 0.19641614+7.95021236e-01j -2.0309045 +4.01238978e-01j
   -1.5076466 -9.20129597e-01j  0.73069614-2.62220085e-01j]
  [ 1.2982327 +6.43608809e-01j -1.6246527 +1.46745992e+00j
   -0.3415263 -1.58099043e+00j -0.02620813+1.40734732e-01j]
  [-1.8381621 -5.22429407e-01j  0.37521768+8.02842319e-01j
    0.65528524+7.47268572e-02j -1.6111596 +9.56368685e-01j]
  [ 0.8975909 +1.92185700e+00j -1.4152474 +1.39655733e+00j
    2.032549  -6.72062755e-01j  1.1926116 +7.93397725e-01j]
  [-0.1267428 +1.28899145e+00j  2.5188825 +2.25052357e+00j
   -2.6430633 +9.63922799e-01j -0.8688984 +2.75514174e+00j]
  [-1.7327905 -1.45382428e+00j -0.7491778 -4.08782721e-01j
   -0.3114716 +1.17125940e+00j -0.22364622-2.97612262e+00j]
  [-1.1442229 +8.58553588e-01j -0.9104696 +1.40404296e+00j
    1.0402938 +2.57935196e-01j -0.9869436 +3.61695439e-01j]
  [-1.1997372 -1.59893680e+00j -0.79166573+7.66955495e-01j
   -1.2